## Create Generate Questions jsonl for finetuned model [ROUGH RN]

** Rough Notebook rn, needs cleanup **

In [1]:
import requests
import json
import transformers
import torch
import re
import pandas as pd

/home/kz9921/.conda/envs/llama_finetuning_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
url = "http://localhost:12257/v1/chat/completions"

headers={
    "Content-Type": "application/json",
    "Authorization": "token-abc123"
}

model_name = 'meta-llama/Meta-Llama-3.1-70B-Instruct'

In [ ]:
# Kylie uses her local scratch model
model_id = '/scratch/gpfs/kz9921/transformer_cache/Llama-3.3-70B-Instruct-bnb-4bit'

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


ValueError: Could not load model /scratch/gpfs/kz9921/transformer_cache/Llama-3.3-70B-Instruct-bnb-4bit with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/home/kz9921/.conda/envs/llama_finetuning_env/lib/python3.11/site-packages/transformers/pipelines/base.py", line 289, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/kz9921/.conda/envs/llama_finetuning_env/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py", line 564, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/kz9921/.conda/envs/llama_finetuning_env/lib/python3.11/site-packages/transform

: 

In [ ]:
def get_question_generation_prompt(justice_name, opening_statement):
    system_prompt = """You are a legal expert trained to simulate the questioning style of Supreme Court justices during oral arguments. Below is an opening statement from a Supreme Court case and the name of a specific justice. Your task is to generate a list of questions that this justice is likely to ask in response to the opening statement. These questions should reflect the justice’s known priorities, jurisprudence, and typical questioning style, and they should be directly relevant to the arguments presented in the opening statement.

        ### Instructions:
        1. Carefully analyze the opening statement to identify key arguments, assumptions, or ambiguities.
        2. Generate questions that the specified justice might ask.
        3. Format each question with specific start and end tags for easier parsing:
           - Use `<START_Q>` at the beginning of each question.
           - Use `<END_Q>` at the end of each question.

        ### Example:
        Justice: Justice Oliver Wendell Holmes Jr.
        Opening Statement: Congress intended a preponderance of the evidence standard to apply to FLSA exemptions. Respondents argue for a clear and convincing evidence standard due to the importance of overtime rights. However, waivability of rights and standards of proof are unrelated.

        Generated Questions:
        <START_Q> Is there any explicit legislative language that supports your claim regarding the preponderance standard? <END_Q>
        <START_Q> How does the principle of statutory construction guide us in resolving ambiguities in this case? <END_Q>
        <START_Q> Would adopting a clear and convincing standard conflict with the broader purpose of the Fair Labor Standards Act? <END_Q>
        <START_Q> How do you reconcile your argument with precedents where the Court imposed heightened standards for "important" rights? <END_Q>

        ### Your Task:
        Justice: {justice_name}
        Opening Statement: {opening_statement}

        ### Output:
        A list of questions that {justice_name} is likely to ask:
        <START_Q> text of question one <END_Q>
        <START_Q> text of question two <END_Q>
        <START_Q> text of question three <END_Q>

    """
    
    user_prompt = f"""### Your Task:
        Justice: {justice_name}
        Opening Statement: {opening_statement}

        ### Output:
    """

    messages = [
            {
                "role": "system",
                "content": system_prompt,
            },
            {"role": "user", "content": user_prompt}
        ]
    # messages = {
    #         "system_prompt": system_prompt,
    #         "instruction": user_prompt
    #         }
    return messages

In [ ]:
def get_model_response(messages):
    output = pipeline(
            messages,
            max_new_tokens=512,
        )
    response = output[0]["generated_text"][-1]
    print(response)
    return response

def parse_response(response):
    decoded = response.content.decode('utf-8')
    response_data = json.loads(decoded)
    content = response_data['choices'][0]['message']['content']
    
    questions = re.findall(r"<START_Q>(.*?)<END_Q>", content, re.DOTALL)
    cleaned_questions = [q.strip() for q in questions]

    return cleaned_questions

: 

#### Read csv with dataframe and try call

In [8]:
input_fp = '../datasets/original/2024_full_text_transcripts.csv'
transcripts_df = pd.read_csv(input_fp)
data = []
current_justices = {"Justice John G. Roberts, Jr.", "Justice Clarence Thomas", "Justice Samuel A. Alito, Jr.", "Justice Sonia Sotomayor", "Justice Elena Kagan", "Justice Neil Gorsuch", "Justice Brett M. Kavanaugh", "Justice Amy Coney Barrett", "Justice Ketanji Brown Jackson"}

def create_justice_questions_tuple(justice, advocate_side, row):
    '''
    Given a justice, advocate to whom the justice is responding, and a row of the 2024
    full text transcripts csv (which contains the transcript id, petitioner opening statement and full text 
    and respondent opening statement and full text), creates and returns a question generation instruction prompt 
    and all relevant data needed to run the prompt effectively.

    @params: justice -- SCOTUS Justice
    @params: advocate_side -- "petitioner" or "respondent"
    @params: row a row of the 2024 full text transcripts csv
    '''
    if advocate_side == "petitioner":
        opening_statement = row['petitioner_opening_text']
    else:
        opening_statement = row['respondent_opening_statement']
    
    sample = get_question_generation_prompt(justice, opening_statement)
    sample.update({
        'justice': justice,
        'question_addressee': advocate_side,
        'opening_statement': opening_statement
    })
    return sample

for justice in current_justices:
    for _, row in transcripts_df.iterrows():
        petitioner_sample = create_justice_questions_tuple(justice, 'petitioner', row)
        data.append(petitioner_sample)
        respondent_sample = create_justice_questions_tuple(justice, 'respondent', row)
        data.append(respondent_sample)

data_df = pd.DataFrame(data)
data_df.to_json('finetune_question_test.jsonl', orient='records', index=False, lines=True)

In [10]:
def add_justice_questions(justice_name, opening_statement):
    messages = get_question_generation_prompt(justice_name, opening_statement)
    response = get_model_response(messages)
    questions = parse_response(response)
    # return json string of list of questions to store in pandas df
    return json.dumps(questions)
    

In [ ]:
for justice in current_justices:
    justice_last_name = justice.split()[-1].lower()
    transcripts_df[f'questions_{justice_last_name}_petitioner'] = transcripts_df.apply(
        lambda row: add_justice_questions(justice, row['petitioner_opening_text']), axis=1)
    transcripts_df[f'questions_{justice_last_name}_respondent'] = transcripts_df.apply(
    lambda row: add_justice_questions(justice, row['respondent_opening_statement']), axis=1)

transcripts_df.head()

ConnectionError: HTTPConnectionPool(host='localhost', port=12257): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x149af8cda890>: Failed to establish a new connection: [Errno 111] Connection refused'))

: 

In [16]:
# add sotomayor, respondent
justice = 'Justice Sonia Sotomayor'
df['questions_sotomayor_respondent'] = df.apply(
    lambda row: add_justice_questions(justice, row['respondent_opening_statement']), axis=1
)
df.head()

,Unnamed: 0,transcript_id,petitioner_opening_text,petitioner_full_text,respondent_opening_statement,respondent_full_text,questions_sotomayor_petitioner,questions_sotomayor_respondent
0,0,2024.23-621-t01,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Anthony A. Yang</speaker><text> Mr. C...,<speaker>Anthony A. Yang</speaker><text> Mr. C...,"[""Petitioner, you argue that a preliminary inj...","[""How do you respond to the argument that a pr..."
1,1,2024.23-365 -t01,<speaker>Lisa S. Blatt</speaker><text> Thank y...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Easha Anand</speaker><text> Mr. Chief...,<speaker>Easha Anand</speaker><text> Mr. Chief...,"[""Counselor, how do you respond to the argumen...","[""Can you explain why the use of \""damages sus..."
2,2,2024.23-852-t01,<speaker>Elizabeth B. Prelogar</speaker><text>...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Peter A. Patterson</speaker><text> Mr...,<speaker>Peter A. Patterson</speaker><text> Mr...,"[""Isn't it true that the proliferation of ghos...","[""Counsel, can you explain why you believe Con..."
3,3,2024.23-980-t01,<speaker>Kannon K. Shanmugam</speaker><text> T...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Kevin K. Russell</speaker><text> Mr. ...,<speaker>Kevin K. Russell</speaker><text> Mr. ...,"[""How do you respond to the argument that the ...","[""Can you explain how your interpretation of t..."
4,4,2024.23-191 -t01,<speaker>Adam G. Unikowsky</speaker><text> Mr....,"<speaker>John G. Roberts, Jr.</speaker><text> ...","<speaker>Edmund G. LaCour, Jr.</speaker><text>...","<speaker>Edmund G. LaCour, Jr.</speaker><text>...","[""Counselor, can you explain how the Alabama S...","[""Can you explain how the distinction between ..."


In [18]:
# # save mid way in case notebook crashes
model_suffix = model_name.split('/')[1]
out_dir = '../datasets/llm_outputs/generate_questions'
out_fp = f'{out_dir}/2024_full_text_sotomayor_questions_{model_suffix}.csv'
df.to_csv(out_fp, index=False)

In [19]:
# add alito, respondent
justice = 'Justice Samuel A. Alito'
df['questions_alito_respondent'] = df.apply(
    lambda row: add_justice_questions(justice, row['respondent_opening_statement']), axis=1
)
df.head()

,Unnamed: 0,transcript_id,petitioner_opening_text,petitioner_full_text,respondent_opening_statement,respondent_full_text,questions_sotomayor_petitioner,questions_sotomayor_respondent,questions_alito_respondent
0,0,2024.23-621-t01,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Anthony A. Yang</speaker><text> Mr. C...,<speaker>Anthony A. Yang</speaker><text> Mr. C...,"[""Petitioner, you argue that a preliminary inj...","[""How do you respond to the argument that a pr...","[""Counselor, can you explain how your interpre..."
1,1,2024.23-365 -t01,<speaker>Lisa S. Blatt</speaker><text> Thank y...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Easha Anand</speaker><text> Mr. Chief...,<speaker>Easha Anand</speaker><text> Mr. Chief...,"[""Counselor, how do you respond to the argumen...","[""Can you explain why the use of \""damages sus...","[""Can you explain how the syntax of the senten..."
2,2,2024.23-852-t01,<speaker>Elizabeth B. Prelogar</speaker><text>...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Peter A. Patterson</speaker><text> Mr...,<speaker>Peter A. Patterson</speaker><text> Mr...,"[""Isn't it true that the proliferation of ghos...","[""Counsel, can you explain why you believe Con...","[""Counselor, you argue that the ATF has exceed..."
3,3,2024.23-980-t01,<speaker>Kannon K. Shanmugam</speaker><text> T...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Kevin K. Russell</speaker><text> Mr. ...,<speaker>Kevin K. Russell</speaker><text> Mr. ...,"[""How do you respond to the argument that the ...","[""Can you explain how your interpretation of t...","[""How does your interpretation of the risk dis..."
4,4,2024.23-191 -t01,<speaker>Adam G. Unikowsky</speaker><text> Mr....,"<speaker>John G. Roberts, Jr.</speaker><text> ...","<speaker>Edmund G. LaCour, Jr.</speaker><text>...","<speaker>Edmund G. LaCour, Jr.</speaker><text>...","[""Counselor, can you explain how the Alabama S...","[""Can you explain how the distinction between ...","[""Can you explain why you think the distinctio..."


In [20]:
# GENERATE FOR ALL
# add alito, petitioner
justice = 'Justice Samuel A. Alito'
df['questions_alito_petitioner'] = df.apply(
    lambda row: add_justice_questions(justice, row['petitioner_opening_text']), axis=1
)
df.head()

,Unnamed: 0,transcript_id,petitioner_opening_text,petitioner_full_text,respondent_opening_statement,respondent_full_text,questions_sotomayor_petitioner,questions_sotomayor_respondent,questions_alito_respondent,questions_alito_petitioner
0,0,2024.23-621-t01,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Anthony A. Yang</speaker><text> Mr. C...,<speaker>Anthony A. Yang</speaker><text> Mr. C...,"[""Petitioner, you argue that a preliminary inj...","[""How do you respond to the argument that a pr...","[""Counselor, can you explain how your interpre...","[""Counselor, how do you respond to the argumen..."
1,1,2024.23-365 -t01,<speaker>Lisa S. Blatt</speaker><text> Thank y...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Easha Anand</speaker><text> Mr. Chief...,<speaker>Easha Anand</speaker><text> Mr. Chief...,"[""Counselor, how do you respond to the argumen...","[""Can you explain why the use of \""damages sus...","[""Can you explain how the syntax of the senten...","[""Can you explain how your interpretation of t..."
2,2,2024.23-852-t01,<speaker>Elizabeth B. Prelogar</speaker><text>...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Peter A. Patterson</speaker><text> Mr...,<speaker>Peter A. Patterson</speaker><text> Mr...,"[""Isn't it true that the proliferation of ghos...","[""Counsel, can you explain why you believe Con...","[""Counselor, you argue that the ATF has exceed...","[""Can you explain how the ATF's new rule addre..."
3,3,2024.23-980-t01,<speaker>Kannon K. Shanmugam</speaker><text> T...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Kevin K. Russell</speaker><text> Mr. ...,<speaker>Kevin K. Russell</speaker><text> Mr. ...,"[""How do you respond to the argument that the ...","[""Can you explain how your interpretation of t...","[""How does your interpretation of the risk dis...","[""Counselor, how does the Ninth Circuit's appr..."
4,4,2024.23-191 -t01,<speaker>Adam G. Unikowsky</speaker><text> Mr....,"<speaker>John G. Roberts, Jr.</speaker><text> ...","<speaker>Edmund G. LaCour, Jr.</speaker><text>...","<speaker>Edmund G. LaCour, Jr.</speaker><text>...","[""Counselor, can you explain how the Alabama S...","[""Can you explain how the distinction between ...","[""Can you explain why you think the distinctio...","[""Can you explain how the Alabama statute at i..."


In [21]:
# save final df
model_suffix = model_name.split('/')[1]
out_dir = '../datasets/llm_outputs/generate_questions'
out_fp = f'{out_dir}/2024_full_text_sotomayor_alito_questions_{model_suffix}.csv'
df.to_csv(out_fp, index=False)

### Try sample call

In [80]:
sample_os = df['petitioner_opening_text'][0]
sample_os

'<speaker>Erika L. Maley</speaker><text> Mr. Chief Justice, and may it please the Court:  The prevailing party is the party who wins the lawsuit, obtaining a final judgment in its favor, or at least a party who obtains a ruling that the defendant is liable on the merits of one or more claims, such as a summary judgment or a judgment as a matter of law. A preliminary injunction is neither a final judgment nor a determination that the defendant is liable on the merits for violating federal law. It is simply a threshold prediction of the likelihood of success based on a truncated record and an initial, often hasty assessment of the law that may well prove to be faulty as the case proceeds. It provid es no enduring relief. By its nature, it is a temporary procedural order that dissolves upon final judgment. A preliminary injunction, therefore, does not make a plaintiff a prevailing party  within the meaning of that legal term of art, and, thus, no statutory exception to the de fault Americ

In [81]:
justice = 'Justice Sonia Sotomayor'

In [82]:
messages = get_question_generation_prompt(justice, sample_os)
messages

[{'role': 'system',
  'content': 'You are a legal expert trained to simulate the questioning style of Supreme Court justices during oral arguments. Below is an opening statement from a Supreme Court case and the name of a specific justice. Your task is to generate a list of questions that this justice is likely to ask in response to the opening statement. These questions should reflect the justice’s known priorities, jurisprudence, and typical questioning style, and they should be directly relevant to the arguments presented in the opening statement.\n\n        ### Instructions:\n        1. Carefully analyze the opening statement to identify key arguments, assumptions, or ambiguities.\n        2. Generate questions that the specified justice might ask.\n        3. Format each question with specific start and end tags for easier parsing:\n           - Use `<START_Q>` at the beginning of each question.\n           - Use `<END_Q>` at the end of each question.\n\n        ### Example:\n    

In [83]:
response = get_model_response(messages)

In [84]:
content = response.content.decode('utf-8')
content

'{"id":"chat-daff385ee8ef45ae8f5659e1844e80a3","object":"chat.completion","created":1732721203,"model":"meta-llama/Meta-Llama-3.1-70B-Instruct","choices":[{"index":0,"message":{"role":"assistant","content":"A list of questions that Justice Sonia Sotomayor is likely to ask:\\n<START_Q> Counselor, doesn\'t the temporary nature of a preliminary injunction actually underscore its importance in protecting the rights of plaintiffs who may not have the resources to see the case through to a final judgment, and wouldn\'t denying them prevailing party status under these circumstances undermine the very purpose of Section 1988? <END_Q>\\n<START_Q> You rely on legal dictionaries from the time of Section 1988\'s enactment, but isn\'t it also relevant to consider the legislative history and the intent behind the statute, which was to encourage the enforcement of civil rights laws and provide a means for plaintiffs to recover attorneys\' fees in doing so? <END_Q>\\n<START_Q> How do you respond to th

In [91]:
questions = parse_response(response)
questions

["Counselor, doesn't the temporary nature of a preliminary injunction actually underscore its importance in protecting the rights of plaintiffs who may not have the resources to see the case through to a final judgment, and wouldn't denying them prevailing party status under these circumstances undermine the very purpose of Section 1988?",
 "You rely on legal dictionaries from the time of Section 1988's enactment, but isn't it also relevant to consider the legislative history and the intent behind the statute, which was to encourage the enforcement of civil rights laws and provide a means for plaintiffs to recover attorneys' fees in doing so?",
 "How do you respond to the argument that adopting a bright-line rule that a preliminary injunction does not make a plaintiff a prevailing party could have a chilling effect on plaintiffs who seek to vindicate their rights through preliminary relief, particularly in cases where the defendant's actions are causing irreparable harm?",
 "Doesn't th

In [117]:
type(questions)
for q in questions:
    print(q)

Counselor, doesn't the temporary nature of a preliminary injunction actually underscore its importance in protecting the rights of plaintiffs who may not have the resources to see the case through to a final judgment, and wouldn't denying them prevailing party status under these circumstances undermine the very purpose of Section 1988?
You rely on legal dictionaries from the time of Section 1988's enactment, but isn't it also relevant to consider the legislative history and the intent behind the statute, which was to encourage the enforcement of civil rights laws and provide a means for plaintiffs to recover attorneys' fees in doing so?
How do you respond to the argument that adopting a bright-line rule that a preliminary injunction does not make a plaintiff a prevailing party could have a chilling effect on plaintiffs who seek to vindicate their rights through preliminary relief, particularly in cases where the defendant's actions are causing irreparable harm?
Doesn't the Court's prec

In [118]:
qs_str = json.dumps(questions)
qs_str

'["Counselor, doesn\'t the temporary nature of a preliminary injunction actually underscore its importance in protecting the rights of plaintiffs who may not have the resources to see the case through to a final judgment, and wouldn\'t denying them prevailing party status under these circumstances undermine the very purpose of Section 1988?", "You rely on legal dictionaries from the time of Section 1988\'s enactment, but isn\'t it also relevant to consider the legislative history and the intent behind the statute, which was to encourage the enforcement of civil rights laws and provide a means for plaintiffs to recover attorneys\' fees in doing so?", "How do you respond to the argument that adopting a bright-line rule that a preliminary injunction does not make a plaintiff a prevailing party could have a chilling effect on plaintiffs who seek to vindicate their rights through preliminary relief, particularly in cases where the defendant\'s actions are causing irreparable harm?", "Doesn\

In [119]:
qs_orig = json.loads(qs_str)
qs_orig

["Counselor, doesn't the temporary nature of a preliminary injunction actually underscore its importance in protecting the rights of plaintiffs who may not have the resources to see the case through to a final judgment, and wouldn't denying them prevailing party status under these circumstances undermine the very purpose of Section 1988?",
 "You rely on legal dictionaries from the time of Section 1988's enactment, but isn't it also relevant to consider the legislative history and the intent behind the statute, which was to encourage the enforcement of civil rights laws and provide a means for plaintiffs to recover attorneys' fees in doing so?",
 "How do you respond to the argument that adopting a bright-line rule that a preliminary injunction does not make a plaintiff a prevailing party could have a chilling effect on plaintiffs who seek to vindicate their rights through preliminary relief, particularly in cases where the defendant's actions are causing irreparable harm?",
 "Doesn't th

In [120]:
type(questions)
for q in questions:
    print(q)

Counselor, doesn't the temporary nature of a preliminary injunction actually underscore its importance in protecting the rights of plaintiffs who may not have the resources to see the case through to a final judgment, and wouldn't denying them prevailing party status under these circumstances undermine the very purpose of Section 1988?
You rely on legal dictionaries from the time of Section 1988's enactment, but isn't it also relevant to consider the legislative history and the intent behind the statute, which was to encourage the enforcement of civil rights laws and provide a means for plaintiffs to recover attorneys' fees in doing so?
How do you respond to the argument that adopting a bright-line rule that a preliminary injunction does not make a plaintiff a prevailing party could have a chilling effect on plaintiffs who seek to vindicate their rights through preliminary relief, particularly in cases where the defendant's actions are causing irreparable harm?
Doesn't the Court's prec